<h3><center>Kinopoisk TOP-250 Movies Page Parsing<center><br>
    using Requests, BeautifulSoup4

Natalia Suchkova - 25.01.2023

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
from lxml import etree

In [48]:
import pandas as pd
import re
import unicodedata

### Тестовый парсинг на одном эксземпляре

In [42]:
url = 'https://www.kinopoisk.ru/lists/movies/top250/?page=1'

In [43]:
r = requests.get(url)
r.text

'<!DOCTYPE html><html><head><meta name="viewport" content="width=device-width"/><meta charSet="utf-8"/><title data-tid="57f72b5">250 лучших фильмов — Кинопоиск</title><meta name="theme-color" content="#1f1f1f" data-tid="57f72b5"/><meta name="apple-mobile-web-app-capable" content="yes" data-tid="57f72b5"/><meta name="apple-mobile-web-app-status-bar-style" content="black" data-tid="57f72b5"/><meta name="apple-mobile-web-app-title" content="Кинопоиск" data-tid="57f72b5"/><meta name="apple-itunes-app" content="app-id=477718890, ct=kp-web, pt=214944, mt=8" data-tid="57f72b5"/><meta name="application-name" content="Кинопоиск" data-tid="57f72b5"/><meta property="fb:app_id" content="121953784483000" data-tid="57f72b5"/><meta property="fb:pages" content="152308956519" data-tid="57f72b5"/><meta property="og:site_name" content="Кинопоиск" data-tid="57f72b5"/><meta name="msapplication-TileColor" content="#000" data-tid="57f72b5"/><meta name="msapplication-TileImage" content="https://yastatic.net/s

In [44]:
soup = BeautifulSoup(r.text, features='lxml')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta content="width=device-width" name="viewport"/>
  <meta charset="utf-8"/>
  <title data-tid="57f72b5">
   250 лучших фильмов — Кинопоиск
  </title>
  <meta content="#1f1f1f" data-tid="57f72b5" name="theme-color"/>
  <meta content="yes" data-tid="57f72b5" name="apple-mobile-web-app-capable"/>
  <meta content="black" data-tid="57f72b5" name="apple-mobile-web-app-status-bar-style"/>
  <meta content="Кинопоиск" data-tid="57f72b5" name="apple-mobile-web-app-title"/>
  <meta content="app-id=477718890, ct=kp-web, pt=214944, mt=8" data-tid="57f72b5" name="apple-itunes-app"/>
  <meta content="Кинопоиск" data-tid="57f72b5" name="application-name"/>
  <meta content="121953784483000" data-tid="57f72b5" property="fb:app_id"/>
  <meta content="152308956519" data-tid="57f72b5" property="fb:pages"/>
  <meta content="Кинопоиск" data-tid="57f72b5" property="og:site_name"/>
  <meta content="#000" data-tid="57f72b5" name="msapplication-TileColor"/>
  <meta content="ht

In [45]:
link = 'https://www.kinopoisk.ru' + soup.find('div', class_='styles_upper__j8BIs').find('a', class_='base-movie-main-info_link__YwtP1').get('href')

In [46]:
link

'https://www.kinopoisk.ru/film/435/'

In [14]:
name = soup.find('div', class_='styles_upper__j8BIs').find('span', class_='styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj').text
name

'Зеленая миля'

In [13]:
name_eng = soup.find('div', class_='styles_upper__j8BIs').find('span', class_='desktop-list-main-info_secondaryTitle__ighTt').text
name_eng

'The Green Mile'

In [71]:
for i in soup.find('div', class_='styles_upper__j8BIs').find_all('span', class_='desktop-list-main-info_truncatedText__IMQRP'):
    print(i.get_text(strip=True))
   # print(unicodedata.normalize("NFKD", i.text))

США • драма  Режиссёр: Фрэнк Дарабонт
В ролях: Том Хэнкс, Дэвид Морс


In [49]:
soup.find('div', class_='styles_upper__j8BIs').find_all('span', class_='desktop-list-main-info_truncatedText__IMQRP')

[<span class="desktop-list-main-info_truncatedText__IMQRP">США • драма  Режиссёр: Фрэнк Дарабонт</span>,
 <span class="desktop-list-main-info_truncatedText__IMQRP">В ролях: Том Хэнкс, Дэвид Морс</span>]

In [112]:
string = unicodedata.normalize("NFKD", soup.find('div', class_='styles_upper__j8BIs').find_all('span', class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
regexp = re.compile("Режиссёр: (.*)$")
director = regexp.search(string).group(1)
director 

'Фрэнк Дарабонт'

In [109]:
string = unicodedata.normalize("NFKD", soup.find('div', class_='styles_upper__j8BIs').find_all('span', class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
regexp = re.compile('(\w+) •')
country = regexp.search(string).group(1)
country 

'США'

In [181]:
string = unicodedata.normalize("NFKD", soup.find('div', class_='styles_upper__j8BIs').find_all('span', class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
regexp = re.compile('• (.*)$')
genre = regexp.search(string).group(1)
regexp = re.compile('[^  ]*')
genre = regexp.search(genre).group(0)
genre

'драма'

In [210]:
string = unicodedata.normalize("NFKD", soup.find('div', class_='styles_upper__j8BIs').find('span', class_='desktop-list-main-info_secondaryText__M_aus').text)
print(string)
year = string.split(', ')[1]
minutes_time = re.compile('[0-9]+').search(string.split(', ')[2]).group(0)
print(year, minutes_time)

, 1999, 189 мин.
1999 189


In [200]:
rating = soup.find('div', class_='styles_upper__j8BIs').find('span', class_='styles_kinopoiskValuePositive__vOb2E styles_kinopoiskValue__9qXjg styles_top250Type__mPloU').text
rating

'9.2'

In [41]:
films = soup.find_all('div', class_='styles_upper__j8BIs')
len(films)

50

In [215]:
data = {}

for film in films:
    link = 'https://www.kinopoisk.ru' + film.find('a', class_='base-movie-main-info_link__YwtP1').get('href')
    name = film.find('span', class_='styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj').text
    print(link, name)
    name_eng = [film.find('span', class_='desktop-list-main-info_secondaryTitle__ighTt').text if film.find('span', class_='desktop-list-main-info_secondaryTitle__ighTt') else '-']
    rating = film.find('span', class_='styles_kinopoiskValuePositive__vOb2E styles_kinopoiskValue__9qXjg styles_top250Type__mPloU').text
    
    string = unicodedata.normalize("NFKD", film.find_all('span', class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
    director = re.compile("Режиссёр: (.*)$").search(string).group(1)
    
    string = unicodedata.normalize("NFKD", film.find_all('span', class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
    country = re.compile('(\w+) •').search(string).group(1)
    
    string = unicodedata.normalize("NFKD", film.find_all('span', class_='desktop-list-main-info_truncatedText__IMQRP')[0].text)
    genre = re.compile('• (.*)$').search(string).group(1)
    genre = re.compile('[^  ]*').search(genre).group(0)
    
    string = unicodedata.normalize("NFKD", film.find('span', class_='desktop-list-main-info_secondaryText__M_aus').text)
    year = string.split(', ')[1]
    minutes_time = re.compile('[0-9]+').search(string.split(', ')[2]).group(0)
    
    current_movie = {'Kinopoisk_Link': link, 'Name': name, 'Name_ENG': name_eng,
                     'Year': year, 'Country': country, 'Genre': genre,
                     'Director': director, 'Length': minutes_time, 'Rating': rating}
    data.update(current_movie)

https://www.kinopoisk.ru/film/435/ Зеленая миля
https://www.kinopoisk.ru/film/329/ Список Шиндлера
https://www.kinopoisk.ru/film/326/ Побег из Шоушенка
https://www.kinopoisk.ru/film/448/ Форрест Гамп
https://www.kinopoisk.ru/film/679486/ Тайна Коко
https://www.kinopoisk.ru/film/3498/ Властелин колец: Возвращение короля
https://www.kinopoisk.ru/film/258687/ Интерстеллар
https://www.kinopoisk.ru/film/476/ Назад в будущее
https://www.kinopoisk.ru/film/342/ Криминальное чтиво
https://www.kinopoisk.ru/film/328/ Властелин колец: Братство Кольца
https://www.kinopoisk.ru/film/312/ Властелин колец: Две крепости
https://www.kinopoisk.ru/film/2360/ Король Лев
https://www.kinopoisk.ru/film/42664/ Иван Васильевич меняет профессию


IndexError: list index out of range